In [4]:
import numpy as np
import pandas as pd
import re
import math
import itertools
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import collections 

from ast import literal_eval
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from collections import namedtuple
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity
from  scipy import spatial
from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale

In [5]:
d2v_model = Doc2Vec.load('../model/word_embedding/Doc2vec_new_small2_4.model')

# modeling

### 데이터셋 불러오기

In [22]:
final_list  = pd.read_excel('./new_data/final_list_score3.xlsx')
inv_list  = pd.read_excel('./new_data/invest_df.xlsx')
final_list = final_list[final_list['established_date2'] < 5.0].reset_index(drop=True)
final_list.fillna('nan', inplace=True)
final_list['token'] = final_list.token.apply(lambda x:literal_eval(x))

In [23]:
final_list['invest'] = [str(x).split(',') for x in final_list['invest']]

In [24]:
invest = list(set([el.strip() for el in final_list['invest'].sum()]))
inv_list.set_index('invest', inplace=True)
inv_list = inv_list.apply(minmax_scale)

In [25]:
inv_list = inv_list.where(inv_list > np.quantile(inv_list['invest_num'], 0.25), np.quantile(inv_list['invest_num'], 0.25))

### Document vector

In [26]:
paragraph_id = {k:v for k, v in enumerate(d2v_model.docvecs.doctags.keys())}
doc_vec = {paragraph_id[k]:list(v) for k,v in enumerate(d2v_model.docvecs.vectors_docs)}

In [36]:
inv_tech_df = pd.DataFrame(columns = ['investor', 'tech', 'weights', 'company'])
for values_list in  final_list[['company', 'predicted_small_label1', 'predicted_small_label2', 'invest']].values:
#     print(values_list)
    for com in values_list[-1]:
#         print(com)
        if com.strip() in ['nan', '비공개', '크라우드펀딩', '사모펀드']:
            continue
        else:
            try:
                weights = inv_list.loc[com.strip()].values[0]
                inv_tech_df = inv_tech_df.append({'investor':com.strip(), 'tech':values_list[1], 'weights':weights, 'company':values_list[0]}, \
                                                 ignore_index=True)
            except KeyError:
                print(com)
                continue

 Indiegogo
이재권
 현대자동차 제로원엑셀러레이터
 신용보증기금 경기스타트업지점


In [37]:
inv = list(inv_list.index)
inv_idx = ["i%d" % i for i in range(1,len(inv_list.index)+1)]
inv_node = pd.DataFrame({'node':inv,'node_idx':inv_idx})

In [86]:
inv_tech_df.rename(columns={'investor':'node'}, inplace=True)

In [87]:
inv_tech_df.to_excel('./new_data/new_experiment/investor_graph2.xlsx', index=False, encoding='cp949')

### 네트워크

In [44]:
def network_score(G) :
    tt =  { "degree":dict(G.degree()), 
            "degree_centrality":nx.degree_centrality(G), #normalized degree, degree가 클수록 높음
            "closeness_centrality":nx.closeness_centrality(G), 
            "betweenness_centrality":nx.betweenness_centrality(G), 
            "pagerank":nx.pagerank(G)}
    tt_df = pd.DataFrame(tt)
    tt_df['node'] = list(tt_df.index)
    tt_df = pd.merge(tt_df,node_df,how='left')
    return tt_df

In [45]:
small_label = list(set(final_list.predicted_small_label1.tolist() + final_list.predicted_small_label2.tolist()))
comb_dv = list(itertools.combinations(small_label,2))

In [46]:
len(small_label)

36

In [47]:
%%time
tech_similarity_list = list()
tech_mat = pd.DataFrame(index=small_label, columns=small_label)
for comb1, comb2 in comb_dv:
    cos_sim = cosine_similarity([doc_vec[comb1]], [doc_vec[comb2]])[0][0]
    tech_mat.loc[comb1,comb2] = cos_sim
    tech_mat.loc[comb2,comb1] = cos_sim
    tech_similarity_list.append(cos_sim)
tech_mat2 = tech_mat.where(tech_mat > 0.13, 0.0)
tech_mat3 = tech_mat2.where(tech_mat2 == 0, 1)

Wall time: 251 ms


In [49]:
t_edge_df = pd.DataFrame(columns= ['source', 'target', 'weights'])
for idx, comb in enumerate(comb_dv):
    edge =tech_mat3.loc[comb[0], comb[1]]
    if edge == 0.0:
        continue
    else:
        weight = tech_mat2.loc[comb[0], comb[1]]
        t_edge_df = t_edge_df.append({'source':comb[0], 'target':comb[1], 'weights':weight}, ignore_index=True)
t_edge = [tuple(element) for element in t_edge_df.values]

In [51]:
%%time
company = list(final_list['company'])
company_idx = ["c%d" % i for i in range(1,len(company)+1)]
node_df = pd.DataFrame({'node':company,'node_idx':company_idx, 'token':final_list.token.tolist()})
sim_sentence = pd.DataFrame(columns=company, index=company)
comb_c = list(itertools.combinations(node_df.node.tolist(),2))
similarity_list = list()
for idx,comb in enumerate(comb_c):
    if idx % 100 == 0:
        print(comb[0], comb[1])
    sent1 = node_df.loc[node_df.node == comb[0], 'token'].values[0]
    sent2 = node_df.loc[node_df.node == comb[1], 'token'].values[0]
    sent1_vector = d2v_model.infer_vector(sent1)
    sent2_vector = d2v_model.infer_vector(sent2)
    sim_sentence.loc[comb[0], comb[1]] = cosine_similarity([sent1_vector], [sent2_vector])[0][0]
    sim_sentence.loc[comb[1], comb[0]] = cosine_similarity([sent2_vector], [sent1_vector])[0][0]
    similarity_list.append(cosine_similarity([sent1_vector], [sent2_vector])[0][0])
sim_sentence2 = sim_sentence.where(sim_sentence > np.quantile(similarity_list, 0.90), 0.0)
sim_sentence3 = sim_sentence2.where(sim_sentence2 == 0, 1)

테라 메디블록
헬스메디 서큘러스
뉴포트파트너스 닥터다이어리
머니브레인 플러스티브이
송아리아이티 메디픽셀
콴텍 캐치잇플레이
스켈터랩스 스테이지랩스
메디픽셀 아모랩
Wall time: 9.55 s


In [53]:
edge_df = pd.DataFrame(columns= ['source', 'target', 'weights'])
for idx,comb in enumerate(comb_c):
    edge =sim_sentence3.loc[comb[0], comb[1]]
    if edge == 0.0:
        continue
    else:
        weight = sim_sentence2.loc[comb[0], comb[1]]
        edge_df = edge_df.append({'source':comb[0], 'target':comb[1], 'weights':weight}, ignore_index=True)
c_edge = [tuple(element) for element in edge_df.values]

In [55]:
small = list(set(list(final_list['predicted_small_label1']) + list(final_list['predicted_small_label2'])))
small_idx = ["s%d" % i for i in range(1,len(small)+1)]
small_node = pd.DataFrame({'node':small,'node_idx':small_idx})
node_df = node_df.append(pd.DataFrame(data=small_node),ignore_index=True)

In [56]:
s_edge = []
for i in range(len(final_list)):
    company = final_list['company'][i]
    small1 = final_list['predicted_small_label1'][i]
    if small1 == 'nan':
        print(company)
    else:
        s_edge.append((small1, company,1))
    small2 = final_list['predicted_small_label2'][i]
    if small2 == 'nan':
        print(company)
    else:
        s_edge.append((small2, company,1))

In [117]:
company_tech = final_list[['company','predicted_label1','predicted_label2', 'predicted_small_label1', 'predicted_small_label2']]
company_tech.rename(columns={'company':'node'}, inplace=True)

C:\Users\lmhoon012\Anaconda3\envs\project\lib\site-packages\pandas\core\frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [59]:
node_df = node_df.append(pd.DataFrame(data=inv_node),ignore_index=True)
node_df.to_excel('./new_data/new_experiment/new_node_df.xlsx', index=False, encoding='cp949')

In [64]:
i_edge = [(el[0], el[3],el[2]) for el in inv_tech_df.values]

### Graph(T-C-I)

In [66]:
nondirect_G = nx.Graph()
nondirect_G.add_weighted_edges_from(c_edge+s_edge+i_edge)
nondirect_score = network_score(nondirect_G)
nondirect_score.to_excel('./new_data/new_experiment/network_score2.xlsx', index=False, encoding='cp949')

#### NTBF

In [123]:
company_network_result = nondirect_score[nondirect_score['node_idx'].isin(company_idx)].sort_values(["pagerank"], ascending=[False])
company_network_result['total_weight'] = company_network_result[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
company_network_result = company_network_result.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
company_network_result = pd.merge(company_network_result, company_tech)
company_network_result.to_excel('./new_data/new_experiment/company_network_result_new.xlsx', index=False, encoding='cp949')

In [218]:
com_score1 = company_network_result.groupby('predicted_small_label1').mean()[['total_weight']].sort_values('total_weight', ascending=False)

In [219]:
com_score2 = company_network_result.groupby('predicted_small_label2').mean()[['total_weight']].sort_values('total_weight', ascending=False)

In [220]:
com_score2.rename(columns ={'total_weight':'total_weight2'}, inplace=True)

In [221]:
total_index = list(set(com_score1.index.tolist() + com_score2.index.tolist()))

In [222]:
company_total_df = pd.DataFrame(index=total_index)

In [223]:
com_score1.index.name = None
com_score2.index.name = None

In [224]:
company_total_df = company_total_df.join(com_score1)
company_total_df = company_total_df.join(com_score2)

In [225]:
company_total_df.fillna(0, inplace=True)

In [226]:
company_score = pd.DataFrame(company_total_df.mean(axis=1), columns=['total_weight']).sort_values('total_weight', ascending=False)

#### Emerging Technology

In [121]:
tech_network_result = nondirect_score[nondirect_score['node_idx'].isin(small_idx)].sort_values(["pagerank"], ascending=[False])
tech_network_result['total_weight'] = tech_network_result[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
tech_network_result = tech_network_result.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
tech_network_result.to_excel('./new_data/new_experiment/tech_network_result_new.xlsx', index=False, encoding='cp949')

In [214]:
tech_network_result[['node', 'total_weight']].sort_values('total_weight', ascending=False).to_excel('emerging_tech.xlsx', index=False)

#### investor

In [215]:
inv_network_result = nondirect_score[nondirect_score['node_idx'].isin(inv_idx)].sort_values(["pagerank"], ascending=[False])
inv_network_result['total_weight'] = inv_network_result[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
inv_network_result = inv_network_result.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
inv_network_result = pd.merge(inv_network_result, inv_tech_df)
inv_network_result.to_excel('./new_data/new_experiment/inv_network_result_new.xlsx', index=False, encoding='cp949')

In [216]:
inv_score = inv_network_result.groupby('tech').mean()[['total_weight']]

In [217]:
inv_score.sort_values('total_weight', ascending=False).to_excel('inv_tech.xlsx')

### Graph1(C-T, C-C)

In [22]:
nondirect_G = nx.Graph()
nondirect_G.add_weighted_edges_from(c_edge+s_edge)
nondirect_score = network_score(nondirect_G)
nondirect_score.to_excel('./new_data/new_experiment/network_score.xlsx', index=False, encoding='cp949')

#### NBTF

In [23]:
company_network_result = nondirect_score[nondirect_score['node_idx'].isin(company_idx)].sort_values(["pagerank"], ascending=[False])
company_network_result['total_weight'] = company_network_result[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
company_network_result = company_network_result.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
company_network_result = pd.merge(company_network_result, company_tech)
company_network_result.to_excel('./new_data/new_experiment/company_network_result.xlsx', index=False, encoding='cp949')

#### Emerging Technology

In [24]:
tech_network_result = nondirect_score[nondirect_score['node_idx'].isin(small_idx)].sort_values(["pagerank"], ascending=[False])
tech_network_result['total_weight'] = tech_network_result[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
tech_network_result = tech_network_result.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
tech_network_result.to_excel('./new_data/new_experiment/tech_network_result.xlsx', index=False, encoding='cp949')

### Graph2(C-T, C-C, T -T)

In [25]:
nondirect_G2 = nx.Graph()
nondirect_G2.add_weighted_edges_from(c_edge+s_edge+t_edge)
nondirect_score2 = network_score(nondirect_G2)
nondirect_score2.to_excel('./new_data/new_experiment/network_score2.xlsx', index=False, encoding='cp949')

#### NBTF

In [31]:
company_network_result2 = nondirect_score2[nondirect_score2['node_idx'].isin(company_idx)].sort_values(["pagerank"], ascending=[False])
company_network_result2['total_weight'] = company_network_result2[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
company_network_result2 = company_network_result2.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
# company_network_result2 = pd.merge(company_network_result2, company_tech)
company_network_result2.to_excel('./new_data/new_experiment/company_network_result2.xlsx', index=False, encoding='cp949')

#### Emerging Technology

In [32]:
tech_network_result2 = nondirect_score2[nondirect_score2['node_idx'].isin(small_idx)].sort_values(["pagerank"], ascending=[False])
tech_network_result2['total_weight'] = tech_network_result2[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
tech_network_result2 = tech_network_result2.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
tech_network_result2.to_excel('./new_data/new_experiment/tech_network_result2.xlsx', index=False, encoding='cp949')

### Graph3 ( I - T)

In [34]:
nondirect_G3 = nx.Graph()
nondirect_G3.add_weighted_edges_from(i_edge)
nondirect_score3 = network_score(nondirect_G3)
nondirect_score3.to_excel('./new_data/new_experiment/investor_network_score.xlsx', index=False, encoding='cp949')

#### Emerging Technology using investor node

In [39]:
invest_network_result = nondirect_score3[nondirect_score3['node_idx'].isin(small_idx)].sort_values(["pagerank"], ascending=[False])
invest_network_result['total_weight'] = invest_network_result[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
invest_network_result = invest_network_result.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
invest_network_result.to_excel('./new_data/new_experiment/invest_network_result.xlsx', index=False, encoding='cp949')

### 네트워크 그리키

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
edge = []
for i in range(len(final_list)):
    weight_sum = 0 
    for j in range(len(final_list['invest'][i])):
        if final_list['invest'][i][j] != 'nan':
            company = final_list['company'][i]
            invest = final_list['invest'][i][j]
            company = node_df[node_df['node']==company]['node_idx'].values[0]
            invest = node_df[node_df['node']==invest]['node_idx'].values[0]
            edge.append((company,invest))

In [ ]:
s_edge = []
for i in range(len(final_list)):
    company = final_list['company'][i]
    small1 = final_list['predicted_small_label1'][i]
    small2 = final_list['predicted_small_label2'][i]
    small1 = node_df[node_df['node']==small1]['node_idx'].values[0]
    small2 = node_df[node_df['node']==small2]['node_idx'].values[0]
    company = node_df[node_df['node']==company]['node_idx'].values[0]
    s_edge.append((small1,company))
    s_edge.append((small2,company))

In [ ]:
s_edge = pd.DataFrame(s_edge,columns=['source','target'])
edge = pd.DataFrame(edge,columns=['source','target'])
df_edges = s_edge.append(edge, ignore_index=True)

In [ ]:
df_nodes = pd.DataFrame()
df_nodes['name'] = node_df['node_idx']

In [ ]:
group = []
for i in range(len(df_nodes)):
    if df_nodes['name'][i][0] == 'c' :
        group.append(0)
    elif df_nodes['name'][i][0] == 's' :
        group.append(1)
    elif df_nodes['name'][i][0] == 'i' :
        group.append(2)     
        
df_nodes['group'] = group

In [ ]:
a = nondirect_score[['node_idx','pagerank']].copy()
a.columns = ['name','nodesize']
df_nodes = pd.merge(df_nodes,a,how = 'left')

In [ ]:
temp = []
for i in range(len(df_nodes)):
    if df_nodes['group'][i] == 0:
        temp.append(df_nodes['nodesize'][i]*10000)
    else :
        temp.append(20)

In [ ]:
df_nodes['nodesize'] = temp

In [ ]:
df_nodes['nodesize'] = df_nodes['nodesize']*5

In [ ]:
color_map = {0:'#ffdf55', 1:'#eebcbc', 2:'#72bbd0'}

In [ ]:
classes = np.array(df_nodes['group'])
classnames, indices = np.unique(classes, return_inverse=True)

In [ ]:
G = nx.Graph(day="Stackoverflow")

for index, row in df_nodes.iterrows():
    G.add_node(row['name'], group=row['group'], nodesize=row['nodesize'])
    
for index, row in df_edges.iterrows():
    G.add_edges_from([(row['source'], row['target'])])

In [ ]:
nx.draw()

In [ ]:
plt.figure(figsize=(25,25))
options = {
    'edge_color': '#FFDEA2',
    'width': 1,
    'with_labels': True,
    'font_weight': 'regular',
}
colors = [color_map[G.nodes[node]['group']] for node in G]
sizes = [G.nodes[node]['nodesize']*10 for node in G]

nx.draw(G, node_color=colors, node_size=sizes, pos=nx.spring_layout(G, k=0.25, iterations=50), **options)
ax = plt.gca()
ax.collections[0].set_edgecolor("#555555") 
plt.show()